We will use the pandas library of python to perform exploratory data analysis. We first read our csv file and store it as a pandas dataframe. Note that we use header=[2] to use the third row as header and skip the first two rows.

In [ ]:
import pandas as pd
d1 = pd.read_csv('./multi_attribute_buying_behaviour.csv', header=[2])
d1.head()

,Attribute_1,Attribute_2,Attribute_3,Buy,Attribute_1.1,Attribute_2.1,Attribute_3.1,Buy.1,Attribute_1.2,Attribute_2.2,Attribute_3.2,Buy.2,Attribute_1.3,Attribute_2.3,Attribute_3.3,Buy.3,Attribute_1.4,Attribute_2.4,Attribute_3.4,Buy.4,Attribute_1.5,Attribute_2.5,Attribute_3.5,Buy.5,Attribute_1.6,Attribute_2.6,Attribute_3.6,Buy.6,Attribute_1.7,Attribute_2.7,Attribute_3.7,Buy.7,Attribute_1.8,Attribute_2.8,Attribute_3.8,Buy.8,Attribute_1.9,Attribute_2.9,Attribute_3.9,Buy.9
0,NaN,NaN,NaN,NaN,No,Yes,Yes,No,Yes,Yes,No,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN,No,Yes,Yes,No,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Yes,Yes,No,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,No,NaN,NaN,NaN,NaN,No,Yes,Yes,No,NaN,NaN,NaN,NaN,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,No,No,Yes,No
2,Yes,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,Yes,No,Yes,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,No,Yes,Yes,No
3,No,Yes,Yes,No,NaN,NaN,NaN,NaN,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,No,Yes,Yes,No,Yes,Yes,NaN,NaN,NaN,NaN,No,Yes,No,No,NaN,NaN,NaN,NaN
4,Yes,Yes,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,Yes,No,NaN,NaN,NaN,NaN,No,No,No,No,NaN,NaN,NaN,NaN,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,Yes,No,Yes,Yes


We see that pandas has labelled the columns with the '.' extension. So the 'Buy' variable for {Store_1, Till_1} remains 'Buy', but that of {Store_1, Till_2} is 'Buy.1'. Similarly, 'Buy' variable for {Store_2, Till_1} is 'Buy.2'. Also note that for any store, an item record is registered as NaN if it was purchased from the other till. 

In total there are 40 columns here - 4 variables x 5 stores x 2 tills per store.

Our final aim of data processessing is to merge item records of all tills and all stores. This will result in a dataframe with only four columns - corresponding to the three attributes and the Buy decision. 

The first step is to merge the data of two tills of a store into one. We iterate over the columns of the dataframe, replace the NaNs with a null string ('') and then concatenate the columns corresponding to any two tills of a store. The new dataframe is stored in a different variable d2.

In [ ]:
columns = ['Attribute_1', 'Attribute_2', 'Attribute_3', 'Buy']
num_stores = 5
d2 = pd.DataFrame()
for col in columns:
  for i in range(num_stores):
    if i == 0:
      d2[col+'.'+str(i)] = d1[col].fillna('') + d1[col+'.1'].fillna('')
    else:
      d2[col+'.'+str(i)] = d1[col+'.'+str(2*i)].fillna('') + d1[col+'.'+str(2*i+1)].fillna('')
d2.head()
      

,Attribute_1.0,Attribute_1.1,Attribute_1.2,Attribute_1.3,Attribute_1.4,Attribute_2.0,Attribute_2.1,Attribute_2.2,Attribute_2.3,Attribute_2.4,Attribute_3.0,Attribute_3.1,Attribute_3.2,Attribute_3.3,Attribute_3.4,Buy.0,Buy.1,Buy.2,Buy.3,Buy.4
0,No,Yes,Yes,Yes,No,Yes,Yes,Yes,No,Yes,Yes,No,Yes,Yes,Yes,No,Yes,Yes,Yes,No
1,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,Yes,No,No,Yes,No
2,Yes,Yes,No,Yes,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,No,Yes,No
3,No,Yes,Yes,Yes,No,Yes,No,Yes,No,Yes,Yes,Yes,No,Yes,No,No,Yes,Yes,Yes,No
4,Yes,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,Yes,No,No,No,Yes,Yes


In the next step, we merge the records for all stores into one. We create subsets of dataframes corresponding to the five stores and then append them below each other.

In [ ]:
num_stores = 5
store_dfs = [pd.DataFrame() for _ in range(num_stores)]
col_ind = 0
for col in columns:
  for store in range(num_stores):
    store_dfs[store][col] = d2.iloc[:, col_ind]
    col_ind+=1







In [ ]:
store_dfs[4].head()
store_dfs[4].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Attribute_1  202 non-null    object
 1   Attribute_2  202 non-null    object
 2   Attribute_3  202 non-null    object
 3   Buy          202 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


Now we append the dataframes corresponding to each store below each other.

In [ ]:
df = store_dfs[0]

for i in range(1,num_stores):
  df = df.append(store_dfs[i])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1010 entries, 0 to 201
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Attribute_1  1010 non-null   object
 1   Attribute_2  1010 non-null   object
 2   Attribute_3  1010 non-null   object
 3   Buy          1010 non-null   object
dtypes: object(4)
memory usage: 39.5+ KB


In [ ]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [ ]:
%R -i df

In [ ]:
%R str(df)

'data.frame':	1010 obs. of  4 variables:
 $ Attribute_1: chr  "No" "Yes" "Yes" "No" ...
 $ Attribute_2: chr  "Yes" "Yes" "No" "Yes" ...
 $ Attribute_3: chr  "Yes" "No" "No" "Yes" ...
 $ Buy        : chr  "No" "Yes" "No" "No" ...


In [ ]:
%R df$Attribute_1 <- as.factor(df$Attribute_1)
%R df$Attribute_2 <- as.factor(df$Attribute_2)
%R df$Attribute_3 <- as.factor(df$Attribute_3)
%R df$Buy <- as.factor(df$Buy)
%R str(df)

'data.frame':	1010 obs. of  4 variables:
 $ Attribute_1: Factor w/ 3 levels "","No","Yes": 2 3 3 2 3 2 2 3 3 3 ...
 $ Attribute_2: Factor w/ 3 levels "","No","Yes": 3 3 2 3 3 3 3 3 3 3 ...
 $ Attribute_3: Factor w/ 3 levels "","No","Yes": 3 2 2 3 3 3 3 2 3 3 ...
 $ Buy        : Factor w/ 3 levels "","No","Yes": 2 3 2 2 2 2 2 3 3 3 ...
